Proof of Concept
===

In [1]:
import PythonLib

## CSV

In [16]:
import pandas as pd

author= "Mark S. Daskin"

dataset="88-nodes.csv"

dataset_file = "./Datasets/" + author + "/" + dataset

data = pd.read_csv(dataset_file)

In [17]:
data

,ID,LATITUDE,LONGITUDE,STATE POP,CITY POP,FIXED COST,CITY
0,1,73.945478,40.670543,7322564,2819401,189600,< New York NY >
1,2,118.411201,34.112101,3485398,1217405,244500,< Los Angeles CA >
2,3,87.684965,41.837050,2783726,1025174,78700,< Chicago IL >
3,4,95.386728,29.768700,1630553,616877,58000,< Houston TX >
4,5,75.134678,40.006817,1585577,603075,49400,< Philadelphia PA >
5,6,117.135770,32.814950,1110549,406096,189400,< San Diego CA >
6,7,83.102198,42.383100,1027974,374057,25600,< Detroit MI >
7,8,96.765249,32.794151,1006877,402060,78800,< Dallas TX >
8,9,112.071399,33.542550,983403,369921,77100,< Phoenix AZ >
9,10,98.505355,29.457650,935933,326761,49700,< San Antonio TX >


## Mongodb

In [18]:
# Generate a connection to our database
from pymongo import MongoClient

mongo_host='ns396089.ip-37-59-38.eu'
mongo_port=32771

client = MongoClient(mongo_host, mongo_port)

# Collection where we wich to store our problem
problems = client.problems[author] # client.[db name].[collection name]

# Prepare the datas to be saved
data.index = data.index.map(str)
record = data.to_dict()

# Insert a new entry to the database
problem_id = problems.insert_one(record).inserted_id

{'ID': {'0': 1,
  '1': 2,
  '2': 3,
  '3': 4,
  '4': 5,
  '5': 6,
  '6': 7,
  '7': 8,
  '8': 9,
  '9': 10,
  '10': 11,
  '11': 12,
  '12': 13,
  '13': 14,
  '14': 15,
  '15': 16,
  '16': 17,
  '17': 18,
  '18': 19,
  '19': 20,
  '20': 21,
  '21': 22,
  '22': 23,
  '23': 24,
  '24': 25,
  '25': 26,
  '26': 27,
  '27': 28,
  '28': 29,
  '29': 30,
  '30': 31,
  '31': 32,
  '32': 33,
  '33': 34,
  '34': 35,
  '35': 36,
  '36': 37,
  '37': 38,
  '38': 39,
  '39': 40,
  '40': 41,
  '41': 42,
  '42': 43,
  '43': 44,
  '44': 45,
  '45': 46,
  '46': 47,
  '47': 48,
  '48': 49,
  '49': 50,
  '50': 51,
  '51': 52,
  '52': 53,
  '53': 54,
  '54': 55,
  '55': 56,
  '56': 57,
  '57': 58,
  '58': 59,
  '59': 60,
  '60': 61,
  '61': 62,
  '62': 63,
  '63': 64,
  '64': 65,
  '65': 66,
  '66': 67,
  '67': 68,
  '68': 69,
  '69': 70,
  '70': 71,
  '71': 72,
  '72': 73,
  '73': 74,
  '74': 75,
  '75': 76,
  '76': 77,
  '77': 78,
  '78': 79,
  '79': 80,
  '80': 81,
  '81': 82,
  '82': 83,
  '83': 84,
  '84